In [1]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import year,col
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DoubleType,TimestampType


In [2]:
spark = SparkSession.builder.appName('aula7').getOrCreate()
df_pix = spark.read.csv('../Aula 7/base_de_dados.csv', header=True, sep=';')

23/10/13 18:11:34 WARN Utils: Your hostname, valdir-G5-5590 resolves to a loopback address: 127.0.1.1; using 192.168.0.54 instead (on interface enp3s0)
23/10/13 18:11:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/13 18:11:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_pix.show()

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+----------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|chave_pix_valor|  data_transacao|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+----------------+
|  1|    9.93|Dra. Ana Carolina...|            79470453|              Nubank|       Maysa da Cruz|             67162333|                 Itau|           cpf|     8439752610|18/02/2022 13:28|
|  2|   15.38|        Ana Caldeira|            19689668|                Itau|        Evelyn Sales|             60005091|             Bradesco|           cpf|    27145380617|08/04/2022 01:47|
|  3|   57.58|    Arthur Goncalves|          

In [4]:
df_pix1 = StructType([
    StructField('id', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('parte_debitada_nome',StringType()),
    StructField('parte_debitada_conta', StringType()),
    StructField('parte_debitada_banco', StringType()),
    StructField('parte_creditada_nome', StringType()),
    StructField('parte_creditada_conta', StringType()),
    StructField('parte_creditada_banco', StringType()),
    StructField('chave_pix_tipo', StringType()),
    StructField('chave_pix_valor', StringType()),
    StructField('data_transacao', TimestampType())
])

In [5]:
df_pix2 = spark.read.csv(
    '../Aula 7/base_de_dados.csv',
    header=True,
    #inferSchema=True,
    sep = ';',
    schema = df_pix1,
    timestampFormat="dd/MM/yyyy HH:mm"
)

In [6]:
df_not_null = df_pix.na.fill(0, subset=['valor']).show()

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+----------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|chave_pix_valor|  data_transacao|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+----------------+
|  1|    9.93|Dra. Ana Carolina...|            79470453|              Nubank|       Maysa da Cruz|             67162333|                 Itau|           cpf|     8439752610|18/02/2022 13:28|
|  2|   15.38|        Ana Caldeira|            19689668|                Itau|        Evelyn Sales|             60005091|             Bradesco|           cpf|    27145380617|08/04/2022 01:47|
|  3|   57.58|    Arthur Goncalves|          

In [7]:

df_pix2.select(year(col('data_transacao'))).show()


+--------------------+
|year(data_transacao)|
+--------------------+
|                2022|
|                2022|
|                2022|
|                2022|
|                2022|
|                2022|
|                2021|
|                2021|
|                2021|
|                2022|
|                2022|
|                2022|
|                2021|
|                2021|
|                2022|
|                2021|
|                2022|
|                2022|
|                2022|
|                2021|
+--------------------+
only showing top 20 rows



In [8]:
df_2022 = df_pix2.filter(year(col('data_transacao')) == '2022')

In [9]:
df_2022.show()

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|chave_pix_valor|     data_transacao|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+
|  1|    9.93|Dra. Ana Carolina...|            79470453|              Nubank|       Maysa da Cruz|             67162333|                 Itau|           cpf|     8439752610|2022-02-18 13:28:00|
|  2|   15.38|        Ana Caldeira|            19689668|                Itau|        Evelyn Sales|             60005091|             Bradesco|           cpf|    27145380617|2022-04-08 01:47:00|
|  3|   57.58|    Arthur Gonca

In [10]:
df_2022.withColumn('year', year(col('data_transacao'))).groupBy('year').count().show()

+----+-----+
|year|count|
+----+-----+
|2022|   48|
+----+-----+



In [13]:
df_pix2.rdd.getNumPartitions()

1

In [ ]:
!lscpu

In [16]:
df_2022.write.mode('overwrite').csv('output/df_2022.csv')

In [19]:
df_2022.write.mode('overwrite').partitionBy('chave_pix_tipo').parquet('output/df_2022.parquet')

In [20]:
df_2022.rdd.getNumPartitions()

1

In [22]:
df_pix.columns

['id',
 'valor',
 'parte_debitada_nome',
 'parte_debitada_conta',
 'parte_debitada_banco',
 'parte_creditada_nome',
 'parte_creditada_conta',
 'parte_creditada_banco',
 'chave_pix_tipo',
 'chave_pix_valor',
 'data_transacao']

In [28]:
from pyspark.sql.functions import date_format


df_final = df_pix2.withColumn('data_particao', date_format(col('data_transacao'),'yyyy-MM-dd'))

In [29]:
df_final.write.mode('overwrite').partitionBy('data_particao').parquet('output/df_2022.parquet')

In [ ]:
df_parquet = spark.read.parquet('output/df_2022.parquet')


In [31]:
# Retorna o numero de partições
df_parquet.rdd.getNumPartitions()

8

In [33]:
# Reduzir o número de partições utilizando o coalesce
df_parquet_coalesce = df_parquet.coalesce(4)

In [34]:
df_parquet_coalesce.rdd.getNumPartitions()

4

In [35]:
df_final_ano_mes = df_pix2.withColumn('data_particao', date_format(col('data_transacao'),'yyyy-MM'))

In [36]:
df_final_ano_mes.write.mode('overwrite').partitionBy('data_particao').parquet('output/df_2022.parquet')